Spot detection: part 1

Data: [OpticalPooledScreens github repository](https://github.com/feldman4/OpticalPooledScreens).

In [7]:
# load data

from skimage import io 

nuclei_url = 'https://raw.githubusercontent.com/kevinyamauchi/napari-spot-detection-tutorial/main/data/nuclei_cropped.tif'
nuclei = io.imread(nuclei_url)

spots_url = 'https://raw.githubusercontent.com/kevinyamauchi/napari-spot-detection-tutorial/main/data/spots_cropped.tif'
spots = io.imread(spots_url)

In [8]:
import napari

# create the napari viewer
viewer = napari.Viewer();

# add the nuclei image to the viewer
viewer.add_image(nuclei);


In [9]:
# add spots to viewer

viewer.add_image(spots)

<Image layer 'spots' at 0x29d75f79220>

Create an image filter
- spots image background/autofl from the cells
- high pass filter to improve contrast of spots


In [10]:
import numpy as np 
from scipy import ndimage as ndi 

def gaussian_high_pass(image: np.ndarray, sigma: float =2):
    """Apply a gaussian high pass filter to an image.
    
    Pramaters
    ----------
    image: np.ndarray
        The image to be filtered
    sigma: float
        The sigma (width) of the gaussian filter to be applied
        Default value = 2 
        
    Returns
    ---------
    high_passed_im : np.ndarray
        The image with high pass filter applied
    """

    low_pass = ndi.gaussian_filter(image, sigma)
    high_passed_img = image - low_pass

    return high_passed_img

In [11]:
# apply gaussian_high_pass function to filter the spots image

filtered_spots = gaussian_high_pass(spots, 2)

# add to layer

viewer.add_image(filtered_spots)

<Image layer 'filtered_spots' at 0x29d73a27040>

Detect Spots

In [12]:
import numpy as np 
from skimage.feature import blob_log

def detect_spots(
    image: np.ndarray,
    high_pass_sigma: float = 2,
    spot_threshold: float = 0.01,
    blob_sigma: float = 2

): 

    """Apply a gaussian high pass to image

    Paramaters
    -----------
    
    image: np.ndarray
        The image to detect spots.
        
    high_pass_sigma: float
        Sigma width of gaussian filter to be applied
        Default value is 2
    spot_threshold: float
        The threshold to be passed by blob detector
        Default value is 0.01
    blob_sigma: float
        The expected sigma (width) of spots. 
        This parameter is passed to the 'max_sigma' parameter of the blob detector
        
    Returns
    --------
    points_coords: np.ndarray
        An NxD array with coordinate for each detected spot
        N is number of spots
        D is the number of dimensions
    sizes: np.ndarray
        An array of size N, where N is the number of detected spots
        with the diameter of each spot.
        
    """

    # filter the image with the gaussian_high_pass filter
    filtered_spots = gaussian_high_pass(image, high_pass_sigma)

    # detect spots on filtered image
    detected_blobs = blob_log(
        filtered_spots,
        max_sigma=blob_sigma,
        num_sigma=1,
        threshold=spot_threshold

    )

    # Convert output of blob detector to desired 
    # points_coords and sizes arrays

    points_coords = detected_blobs[:, 0:2]
    sizes = 3 * detected_blobs[:, 2]

    return points_coords, sizes

In [14]:
# detect spots

spot_coords, spot_sizes = detect_spots(
    spots,
    high_pass_sigma=2,
    spot_threshold=0.01,
    blob_sigma=2
)

In [16]:
# add detected spots to the viewer as a point layer
viewer.add_points(
    spot_coords,
    size=spot_sizes,
    opacity=0.5
)

viewer.add_shapes

<bound method add_shapes of Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 245.5, 246.5), zoom=1.5485772357723575, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(273.2674312667802, 177.40432366745708), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 492.0, 1.0), (0.0, 494.0, 1.0)), current_step=(246, 247), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'nuclei' at 0x29d5f900f10>, <Image layer 'spots' at 0x29d75f79220>, <Image layer 'filtered_spots' at 0x29d73a27040>, <Points layer 'spot_coords' at 0x29d73f6e190>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<Tex